# Imports

In [1]:
import torch, numpy as np

from torch.utils.data import DataLoader

from datasets import Dataset, load_dataset

import IPython
import IPython.display
import julius
import librosa

import time


import torchaudio
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps


import os
import uuid


import whisper
import os


c:\Users\Danik\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


# Load datasets

[ivrit.ai](https://huggingface.co/datasets/ivrit-ai/crowd-transcribe-v5) </br>
[HebDB](https://huggingface.co/datasets/SLPRL-HUJI/HebDB) </br>

Might use [Hebrew speech Kan](https://huggingface.co/datasets/imvladikon/hebrew_speech_kan) and [Shaul's dataset](https://openslr.org/134/)


In [2]:
data = load_dataset('ivrit-ai/crowd-transcribe-v5')
print(data)

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/39 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['uuid', 'audio', 'orig_sentence', 'sentence', 'is_retranscribe', 'transcriber', 'extra_data'],
        num_rows: 203827
    })
    test: Dataset({
        features: ['uuid', 'audio', 'orig_sentence', 'sentence', 'is_retranscribe', 'transcriber', 'extra_data'],
        num_rows: 20711
    })
})


In [3]:
torch_train = data['train'].with_format("torch")
torch_test = data['test'].with_format("torch")

In [4]:
torch_train[0]

{'uuid': 'Tziun3/2021.06.03 פרק #207 – חזרתו של היוני/343',
 'audio': {'path': '343.mp3',
  'array': tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0033, -0.0035, -0.0037]),
  'sampling_rate': tensor(44100)},
 'orig_sentence': 'לא אני קטונתי כן אבל אני במקומו הייתי הולך על מהלך אמרתי לך ללכת כזה לאקדמיה של',
 'sentence': 'לא, זה לא אני קטונתי כן אבל אני במקומו הייתי הולך על מהלך אמרתי לך ללכת כזה לאקדמיה של',
 'is_retranscribe': tensor(False),
 'transcriber': tensor(0),
 'extra_data': {'attributes': None,
  'duration': tensor(6.3478),
  'foreign_language': tensor(False),
  'max_logprob': tensor(-0.2381),
  'multiple_speakers': tensor(False),
  'noisy': tensor(False),
  'orig_text': 'לא אני קטונתי כן אבל אני במקומו הייתי הולך על מהלך אמרתי לך ללכת כזה לאקדמיה של',
  'skipped': tensor(False),
  'stats': None,
  'text': 'לא, זה לא אני קטונתי כן אבל אני במקומו הייתי הולך על מהלך אמרתי לך ללכת כזה לאקדמיה של',
  'too_long': tensor(False),
  'unintelligible': tensor(False)}}

# Resample to 16KHz
They did this using julius, librosa runs 200x faster and sounds the fucking same </br>
Julius also sucks!! This is straight shit!! I do not know why are there so many methods, and which one to use. </br></br>
Librosa is more relatable like "yo bro you wanna resample? I gotchu!!! use my <b>RESAMPLE</b> method". </br>
Julius is a dickhead it's like "Oh yeah you should analyze which specific function from within my 40000 modules you want".

In [5]:
for i in range(20000):
    if torch_test[i]['extra_data']['duration'].item() > 20:
        break
i

123

In [6]:
audio = torch_test[123]['audio']['array']
sr = torch_test[123]['audio']['sampling_rate']

In [7]:
bef = time.time()

res_julius = julius.resample._downsample2(audio, 16000)

aft = time.time()

print('Resampling took ' + str(aft-bef) + ' seconds')
IPython.display.Audio(res_julius, rate=16000)

Resampling took 76.67678308486938 seconds


In [8]:
bef = time.time()

res_librosa = librosa.resample(audio.detach().numpy(), orig_sr=sr, target_sr=16000)

aft = time.time()

print('Resampling took ' + str(aft-bef) + ' seconds')
IPython.display.Audio(res_librosa, rate=16000)

Resampling took 0.005955219268798828 seconds


# Perform VAD and split data to short segments

In [9]:
model = load_silero_vad()

speech_timestamps = get_speech_timestamps(
  res_librosa,
  model,
  return_seconds=True,
)

speech_timestamps

[{'start': 0.0, 'end': 7.2},
 {'start': 7.3, 'end': 17.0},
 {'start': 17.2, 'end': 28.0},
 {'start': 28.1, 'end': 29.0}]

In [10]:
MIN_DURATION = 1.0  # seconds
SILENCE_GAP = 0.1   # 100 ms
PAD = 0.03          # 30 ms

sr = 16000

valid_segments = []
for seg in speech_timestamps:
    duration = seg['end'] - seg['start']

    if duration >= MIN_DURATION:
        valid_segments.append(seg)

# You also may want to merge segments that are closer than SILENCE_GAP
merged_segments = []
for seg in valid_segments:
    if not merged_segments:
        merged_segments.append(seg)
    else:
        prev = merged_segments[-1]
        # If current segment starts within the SILENCE_GAP from the previous segment’s end, merge them
        if seg['start'] - prev['end'] < SILENCE_GAP:
            merged_segments[-1]['end'] = seg['end']
        else:
            merged_segments.append(seg)


merged_segments

[{'start': 0.0, 'end': 17.0}, {'start': 17.2, 'end': 28.0}]

In [11]:
pad_samples = int(PAD * sr)  # Number of samples for 30ms
silence = torch.zeros(pad_samples)  # Shape: (1, pad_samples) for mono

output_dir = "segmented_wav"
os.makedirs(output_dir, exist_ok=True)

for i, seg in enumerate(merged_segments):
    start_sample = int(seg['start'] * sr)
    end_sample   = int(seg['end'] * sr)

    segment_waveform = torch.cat((silence, torch.tensor(res_librosa[start_sample:end_sample]), silence))

    if segment_waveform.ndim == 1:
        segment_waveform = segment_waveform.unsqueeze(0)

    torchaudio.save(output_dir+f'/segment_{i}.wav', segment_waveform, sr)


# Transcribe each segment using Whisper V2 large
This is a large model (1.5B params) but we can run it now uni servers. We only need some inference. </br>
On my CPU (12th gen intel i5-12400) inference took $\approx45$ sec per audio segment ($\approx12$ sec audio), so maybe we have hope of finishing this shit quickly on Uni GPUs!

In [12]:
model = whisper.load_model("large-v2")  # loads the Whisper Large V2 model

In [13]:
segmented_dir = "segmented_wav"

results = []

for filename in os.listdir(segmented_dir):
    if filename.endswith(".wav"):
        audio_path = os.path.join(segmented_dir, filename)

        audio, sr = librosa.load(audio_path, sr=16000, mono=True)

        result = model.transcribe(audio, language="he")  # specify Hebrew if needed

        results.append({filename : result['text']})

c:\Users\Danik\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [14]:
for res in results:
    print(res, end = 3*'\n')

{'segment_0.wav': ' לא מוזכרת כשפה שצריך תרגם אליה את המניפסט. שלא הם יחשבו עליה בתור אירופאית. לא, לא, כנראה שלא. אז באמת, אני רוצה שעוד מעט באמת ניכנס ל... כי בכל זאת, אתה פילוסופ, אני רוצה שתדבר קצת על העקרונות. אני מורה לפילוסופיה.'}


{'segment_1.wav': ' מה זה מורה לפילוסופיה אם לא פילוסוף שעוסק בהוראה. אני מחמיא לך, מה איך אמרת? לא, זה ממש. אז באמת לפני אבל שאנחנו צוללים לזה, מתי המניפסט נהיה מניפסט? כלומר, מתי קומוניסטים נהיו...'}


